In [ ]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


This code example is similar to v5_9_text_autocompletion, but it works on words (encoded with an embedding layer) instead of characters and it does not do beam search. More context for this code example can be found in video 6.4 "Programming Example: Language Model and Word Embeddings with PyTorch" in the video series "Learning Deep Learning: From Perceptron to Large Language Models" by Magnus Ekman (Video ISBN-13: 9780138177614).


The initialization code below contains a couple of additional imports compared to v5_9_text_autocompletion. It might seem odd that we are importing modules from TensorFlow given that this code example is supposed to be implemented in PyTorch. This is not a typo. We simply decided to use some functionality from TensorFlow for preprocessing text. The DL model itself is fully implemented in PyTorch.

We also define two new constants MAX_WORDS and EMBEDDING_WIDTH that define the max size of our vocabulary and the dimensionality of the word vectors.


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
# Using Keras Tokenizer for simplicity
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text \
    import text_to_word_sequence
import numpy as np
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = '../data/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 7500
EMBEDDING_WIDTH = 100


The next code snippet first reads the input file and splits the text into a list of individual words. The latter is done by using the imported function text_to_word_sequence(), which also removes punctuation and converts the text to lowercase, so we do not need to do that manually in this example. We then create input fragments and associated target words just as in the character-based example.


In [ ]:
# Open the input file.
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8-sig')
text = file.read()
file.close()

# Make lower case and split into individual words.
text = text_to_word_sequence(text)

# Create training examples.
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])


The next step is to convert the training examples into the correct format. Each input word needs to be encoded to a corresponding word index (an integer). This index will then be converted into an embedding by the Embedding layer. The target (output) word should still be one-hot encoded. To simplify how to interpret the output, we want the one-hot encoding to be done in such a way that bit N is hot when the network outputs the word corresponding to index N in the input encoding.

We make use of the Tokenizer class. When we construct our tokenizer, we provide an argument num_words = MAX_WORDS that caps the size of the vocabulary. The tokenizer object reserves index 0 to use as a special padding value and index 1 for unknown words. The remaining 7,498 indices (MAX_WORDS was set to 7,500) are used to represent words in the vocabulary.

The padding value (index 0) can be used to make all training examples within the same batch have the same length. The Embedding layer can be instructed to ignore this value, so the network does not train on the padding values.

Index 1 is reserved for UNKnown (UNK) words because we have declared UNK as an out-of-vocabulary (oov) token. When using the tokenizer to convert text to tokens, any word that is not in the vocabulary will be replaced by the word UNK. Similarly, if we try to convert an index that is not assigned to a word, the tokenizer will return UNK. If we do not set the oov_token parameter, it will simply ignore such words/indices.

After instantiating our tokenizer, we call fit_on_texts() with our entire text corpus, which will result in the tokenizer assigning indices to words. We can then use the function texts_to_sequences to convert a text string into a list of indices, where unknown words will be assigned the index 1.


In [ ]:
# Convert to indices.
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

# Convert to appropriate input and output formats.
X = np.array(fragments_indexed, dtype=np.int64)
y = np.zeros(len(targets_indexed), dtype=np.int64)
for i, target_index in enumerate(targets_indexed):
    y[i] = target_index[0]


Just like in the previous example, we split the data into a training set and a test set and create Dataset objects.

In [ ]:
# Split into training and test set.
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.05, random_state=0)

# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
testset = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))


We are now ready to create the model, but we do this differently than we have done in previous examples, both to illustrate some new constructs but also to provide more flexibility in what we are building. Instead of instantiating a Sequential model with standard layers, we build a fully customized model. This is done by extending the nn.Module class. We already saw examples of that when we built custom layers, and we use a similar methodology when building a fully custom model.

To understand the details of this model we first need to take a detour and discuss some details of how we plan to use the model to do predictions. We do this a little bit differently than in the previous example. Instead of feeding a string of symbols as input to the model, we feed it only a single symbol at a time. This is an alternative implementation compared to the previous implementation, where we repeatedly fed the model a growing sequence of characters.

The scheme used in this example has a subtle implication, which has to do with dependencies between multiple consecutive calls to the model. In the previous example we did not have an expectation that the inputs to the first prediction should impact the second prediction. In this example, we do want that to happen. We want the LSTM layers to retain their c and h states from one call to another so that the outputs of subsequent calls to the model will depend on the prior calls to the model. We provide this functionality with our own custom methods set_state(), get_state(), and clear_state().

We are now ready to describe the details of our model. The __init__() function will be called a single time when the model is created. When building a custom model, this is the place to put any initialization code, such as declaring the layers to be used by the model as well as any other variables that will be needed.

We start with declaring two variables state and use_state to control how we handle initialization of the internal state of the LSTM layers (see below).

For the model, we start with declaring an Embedding layer with MAX_WORDS as input size and EMBEDDING_WIDTH as output size. We adjust the weight initialization to use uniform random numbers between -0.05 and 0.05, as opposed to the default range of -1.0 to 1.0. We did this to match the range used in our TensorFlow examples. 

Next we declare an LSTM module containing two LSTM layers. We also add a Dropout layer, followed by two fully connected layers, with ReLU activation in-between.

The behavior of the model is defined in the forward method. We feed the inputs to the embedding layer and the resulting outputs are temporarily stored in the variable x. Next we arrive at the custom state handling code for the LSTM layers. If our variable use_state is False (the value we initialized it to), we will call the LSTM layers with just x as input. This implies that the LSTM module will use 0 as initial h and c states. However, if use_state is set to true, we will also supply the variable self.state as input to the LSTM layers. In that case the LSTM layers will use these states as their initial states instead of 0.

After the LSTM layers has been called, we retrieve the resulting internal state and store it in self.state so it can be used as input for the next timestep. Apart from calling detach() we also call clone(), which makes a copy of the state so it does not change under the hood by the layers themselves if they are later called with new inputs.

We then feed the output from the top LSTM layer through the Dropout layer. The indices after variable x results in selecting the final timestep for the top LSTM layer. That is, this is equivalent to the functionality of the custom layer in c11e1_autocomplete. Given that we are building a custom model and have explicit control of how the layers are connected, there is no need to declare a custom layer.

The rest of the forward method is straight forward and we feed the output from the Dropout layer through a Linear layer, followed by a ReLU layer, and finally another Linear layer that represents the Softmax layer but without its activation function, which is later included in the loss function.

Now let us detail the intended use of set_state, get_state, and clear_state. At the beginning of time the use_state variable is set to False. When calling the model, it will use 0 as internal state. After each call to the model, we can retrieve the resulting state by calling get_state. If we later want the model to use this state as its initial state, we simply call set_state and supply that state to the model, before we call the model with its input data. If we later decide that we want to start with a cleared internal state again, we simply call the clear_state method. This concludes the description of the custom model.

Next we instantiate the model, select our normal optimizer and loss function, and finally train the model. Our training process does not make use of the just described state handling and the model will simply use 0 as initial state for each call to the model during training.


In [ ]:
# Define model.
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.state = None
        self.use_state = False
        self.embedding_layer = nn.Embedding(MAX_WORDS, EMBEDDING_WIDTH)
        nn.init.uniform_(self.embedding_layer.weight, -0.05, 0.05) # Default is -1, 1.
        self.lstm_layers = nn.LSTM(EMBEDDING_WIDTH, 128, num_layers=2, dropout=0.2, batch_first=True)
        self.dropout_layer = nn.Dropout(0.2)
        self.linear_layer = nn.Linear(128, 128)
        self.relu_layer = nn.ReLU()
        self.output_layer = nn.Linear(128, MAX_WORDS)

    def forward(self, inputs):
        x = self.embedding_layer(inputs)
        if(self.use_state):
            x = self.lstm_layers(x, self.state)
        else:
            x = self.lstm_layers(x)
        self.state = (x[1][0].detach().clone(), x[1][1].detach().clone()) # Store most recent internal state.
        x = self.dropout_layer(x[1][0][1])
        x = self.linear_layer(x)
        x = self.relu_layer(x)
        x = self.output_layer(x)
        return x

    # Functions to provide explicit control of LSTM state.
    def set_state(self, state):
        self.state = state
        self.use_state = True
        return

    def get_state(self):
        return self.state

    def clear_state(self):
        self.use_state = False
        return

model = LanguageModel()

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

# Train the model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'acc')


After training the model, we are ready to use it to do predictions. The first loop in the code snippet below feeds an initial sequence of words to the model. Note how we call get_state after each call followed by set_state, which instructs the model to use this stat instead of reseting it before each word.

The second loop contains the autoregression logic where we identify the word that the model predicts as highest probability and then feed that as input to the model in the next timestep. To simplify the implementation, we do not do beam search this time around but simply predict the most probable word at each timestep.

We conclude with printing the resulting autocompleted text sequence.


In [ ]:
# Provide beginning of sentence and
# predict next words in a greedy manner.
first_words = ['i', 'saw']
first_words_indexed = tokenizer.texts_to_sequences(
    first_words)
model.clear_state()
predicted_string = ''
# Feed initial words to the model.
for i, word_index in enumerate(first_words_indexed):
    x = np.zeros((1, 1), dtype=np.int64)
    x[0][0] = word_index[0]
    predicted_string += first_words[i]
    predicted_string += ' '
    inputs = torch.from_numpy(x)
    inputs = inputs.to(device)
    outputs = model(inputs)
    y_predict = outputs.cpu().detach().numpy()[0]
    state = model.get_state()
    model.set_state(state)
# Predict PREDICT_LENGTH words.
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts(
        [[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    inputs = torch.from_numpy(x)
    inputs = inputs.to(device)
    outputs = model(inputs)
    y_predict = outputs.cpu().detach().numpy()[0]
    state = model.get_state()
    model.set_state(state)
print(predicted_string)


All of the preceding code had to do with building and using a language model. The next code snippet adds some functionality to explore the learned embeddings. We first read out the word embeddings from the Embedding layer by accessing the weight variable from the model's first layer, which represents the Embedding layer. We move the weights back to the CPU and convert them to NumPy format. We first have to call detach() and clone().

We then declare a list of a number of arbitrary lookup words. This is followed by a loop that does one iteration per lookup word. The loop uses the Tokenizer to convert the lookup word to a word index, which is then used to retrieve the corresponding word embedding. The Tokenizer functions are generally assumed to work on lists. Therefore, although we work with a single word at a time, we need to provide it as a list of size 1, and then we need to retrieve element zero ([0]) from the output.

Once we have retrieved the corresponding word embedding, we loop through all the other embeddings and calculate the Euclidean distance to the embedding for the lookup word using the NumPy function norm(). We add the distance and the corresponding word to the dictionary word_indices. Once we have calculated the distance to each word, we simply sort the distances and retrieve the five word indices that correspond to the word embeddings that are closest in vector space. We use the Tokenizer to convert these indices back to words and print them and their corresponding distances.


In [ ]:
# Explore embedding similarities.
it = model.modules()
next(it)
embeddings = next(it).weight
embeddings = embeddings.detach().clone().cpu().numpy()

lookup_words = ['the', 'of']
for lookup_word in lookup_words:
    lookup_word_indexed = tokenizer.texts_to_sequences(
        [lookup_word])
    print('words close to:', lookup_word)
    lookup_embedding = embeddings[lookup_word_indexed[0]]
    word_indices = {}
    # Calculate distances.
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(
            embedding - lookup_embedding)
        word_indices[distance] = i
    # Print sorted by distance.
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')
